# INFO-F-422 -  Statistical Foundations of Machine Learning 

### Alexandre Flachs - __[alexandre.flachs@ulb.be](mailto:alexandre.flachs@ulb.be) - Student ID 474748__
### Marie Giot - __[marie.giot@ulb.be](mailto:marie.giot@ulb.be) - Student ID 474915__
### Jeanne Szpirer - __[jeanne.szpirer@ulb.be](mailto:jeanne.szpirer@ulb.be) - Student ID 477286__

### Video presentation: www.youtube.com/abcd1234

## Flu Shot Learning: Predict H1N1 and Seasonal Flu Vaccines


# Introduction


*Ajouter du texte d'intro avec jolies images ?* ?

# Data preprocessing

Before working any model we need to preprocess the data to make it usefull. This pipeline in divided intro three parts :
1. **Missing value imputation** : Replace missing values, possibly using other known values
2. **Feature engineering** : Define useful features from available ones. 
3. **Feature selection** : Some features might be useless or give wrong indications to the model, we might need to remove some features.

Let's start by importing our data, then develop each of the above parts.

In [ ]:
# Training set features
training_set_features <- read.csv("training_set_features.csv", stringsAsFactors = T, na.strings = c("NA", ""))
dim(training_set_features)

# Test set features
test_set_features <- read.csv("test_set_features.csv", stringsAsFactors = T, na.strings = c("NA", ""))
dim(test_set_features)

# Training set labels
training_set_labels <- read.csv("training_set_labels.csv", stringsAsFactors = T, na.strings = c("NA", ""))
dim(training_set_labels)

We can see that the training features set and the training labels set has the same amount of lines, this is a first good sign because it means that we have an "answer" for every training line.

## Missing value imputation


We summarize our data before doing any work.

In [ ]:
summary(training_set_features)

We see that we have many missing values, in most features. We can compare the number of lines left if we remove any line containing any missing value.

In [ ]:
# First method
cat("Training set : ", dim(training_set_features)[1], "->", dim(na.omit(training_set_features))[1], "\n")
cat("Test set     : ", dim(test_set_features)[1], "->", dim(na.omit(test_set_features))[1], "\n")
cat("Training labs: ", dim(training_set_labels)[1], "->", dim(na.omit(training_set_labels))[1])


At least no line from the training labels misses any value, we can thus use every entry from the training set for both targets.
Counting the number of missing value per feature allows us to see if some of the could be useless. The health insurance, employment occupation and employment industry lines are the emptiest (almost half of the lines miss this data) but by intuition this might be a huge factor in the vaccination decision so we keep it for now.

In [ ]:
# On peut aussi regarder si certaines colonnes n'ont vraiment quasi aucune valeur, dans ce cas, ça vaut pas vraiment la peine de garder
a <- sapply(training_set_features, function(x) sum(is.na(x)))
print(a[order(a, decreasing=T)])

What happens if we remove these fields ?

In [ ]:
useful_nas <- which(sapply(test_set_features, function(x) sum(is.na(x))) > 10000)

cat("Training set : ",
    dim(training_set_features[,-useful_nas])[1], "->",
    dim(na.omit(training_set_features[,-useful_nas]))[1], "\n")
cat("Test set     : ", dim(test_set_features[,-useful_nas])[1], "->",
    dim(na.omit(test_set_features[,-useful_nas]))[1], "\n")
cat("Training labels: ", dim(training_set_labels)[1], "->", dim(na.omit(training_set_labels))[1])


There are much less lines containing missing values. We will thus manage the three most missed fields differently.

## Imputation of missing numerical values

In [ ]:
# Imputation will be the same for training and testing so we merge them
features_set <- rbind(training_set_features, test_set_features)


# Note the indexes of each dataset to get them back after
tr_indexes <- 1:nrow(training_set_features)
ts_indexes <- (nrow(training_set_features)+1):(nrow(training_set_features) + nrow(test_set_features))


### Integer encoding of some variables

In [ ]:
levels(features_set[, "age_group"])
levels(features_set[, "age_group"]) <- 0:4
features_set[, "age_group"] <- as.numeric(features_set[, "age_group"])
features_set[, "age_group"] <- (features_set[, "age_group"] - 1)/4

levels(features_set[, "education"])
levels(features_set[, "education"]) <- 0:3
features_set[, "education"] <- as.numeric(features_set[, "education"])
features_set[, "education"] <- (features_set[, "education"] - 1)/3

levels(features_set[, "income_poverty"])
levels(features_set[, "income_poverty"]) <- c(1, 2, 0)
features_set[, "income_poverty"] <- as.numeric(features_set[, "income_poverty"])
features_set[, "income_poverty"] <- (features_set[, "income_poverty"] - 1)/2

levels(features_set[, "census_msa"])
levels(features_set[, "census_msa"]) <- c(1, 2, 0)
features_set[, "census_msa"] <- as.numeric(features_set[, "census_msa"])
features_set[, "census_msa"] <- (features_set[, "census_msa"] - 1)/2


### Non negative matrix factorization based imputation

Non negative matrix factorization is a procedure which consists of approximating a matrix with non negative values as the product of two matrices having smaller dimensions. More formally, a matrix $M$ of dimension $m\times n$ is decomposed into two matrices $W$ and $H$ of dimensions $m\times p$ and $p\times n$ respectively such that $M \approx WH$.

More details concerning the imputation method can be found here https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8510447/

PseudoCode :

```pseudocode
Consider M of dim m x n having missing values
Fix N > 0
Normalize M and replace NAs using mean imputation
k1 <- floor(max(abs(rank(M) - N/2), 1))
kN = min(abs(rX + N/2), m, n)

# Compute approximation
initialize X array of size kN-k1+1
for k in 1...kN-k1+1 do
    Compute the nNMF of M on non missing values and store the result in X[k]

## Based on all X[k], reconstruct M
# Weights based only on non missing values of M
initialize d array of size kN-k1+1
for k in 1...kN-k1+1 do
    Compute d[k] = sum(X_ij - M_ij)/ nb_nonNA

# Reconstruct
Initialize divisor=0, M_hat of 0 with same dim as M
for k in 1...kN-k1+1 do
    M_hat += exp(-d[k])*X[k]
    denum += exp(-d[k])
M_hat <- M_hat/denum
```


After these steps, M_hat does not contain missing values and the imputation is based on local and global information.

In [ ]:
library(NMFN)

# We work only on the columns of numeric type
numeric_variables_idx<-which(sapply(features_set[1,],class)!="factor")
numeric_variables_idx <- numeric_variables_idx[-c(1, 16)] # Remove resp ID and health insur

# We need to normalize this data to use efficient nNMF
library("caret")

ss <- preProcess(as.data.frame(features_set[, numeric_variables_idx]), method=c("range"))
features_set[,numeric_variables_idx] <- predict(ss, as.data.frame(features_set[, numeric_variables_idx]))
summary(features_set[, numeric_variables_idx])


In [ ]:
# Define the function that will compute nmf
nfm_mult_upd <- function(R, K, missing_idx, maxit=800, eps=2.2204e-16) {
    # Using weighted multiplicative rule Zhu 2016
    # init random W and H
    print(paste("[INFO] : NMF with k=", K))
    R <- as.matrix(R)
    I <- dim(R)[1]
    J <- dim(R)[2]
    M <- matrix(1, nrow = dim(R)[1], ncol = dim(R)[2])
    M[missing_idx] <- 0
    X <- R # Store original R
    R <- R*M
    W <- matrix(runif(I*K), nrow = I, ncol = K)
    H <- matrix(runif(K*J), nrow = K, ncol = J)
    
    n <- 0
    d1 <- 1000
    d2 <- 1000
    while(n < maxit && !(d1 < eps && d2 < eps)) {
        if (n %% 100 == 0) {
            print(paste("[INFO] : iter", n, " Relative error is :", distance2(X, W%*%H)/distance2(X, R*0)))
        }
        newH <- H* (t(W) %*% R) / (t(W) %*% W %*% H)
        newW <- W*(R %*% t(newH)) / ((W %*% newH) %*% t(newH))
        
        d1 <- distance2(newH, H)
        d2 <- distance2(newW, W)
        
        H <- newH
        W <- newW
        n <- n+1
    }
    
    Res <- W%*%H
    #Res[missing_idx] <- X[missing_idx]
    nmf <- list("res"=Res, "dst"=distance2(R, Res)/distance2(R, 0))
    return(nmf)
}


In [ ]:
# 1: initialize by defining N and replace NAs by mean
N <- 4

replace_na_with_mean_value<-function(vec) {
    mean_vec <- mean(as.numeric(vec), na.rm=TRUE)
    vec[is.na(vec)]<-mean_vec
    vec
}

X<-data.frame(apply(features_set[, numeric_variables_idx], MARGIN=2, replace_na_with_mean_value))
miss_idx = which(is.na(features_set[,numeric_variables_idx]), arr.ind = T)
non_miss_idx <- which(!is.na(features_set[,numeric_variables_idx]), arr.ind = T)



In [ ]:
# 2: nNMF for k=k1 to K=kN with masking missing values
library(Matrix) # For rankMatrix 
rX <- rankMatrix(X)
k1 = floor(max(abs(rX - N/2), 1))
kN = min(abs(rX + N/2), dim(X)[1], dim(X)[2])

X_hat = array(0, dim = c(kN-k1+1, nrow(X), ncol(X)))
dim(X_hat)
for (K in k1:kN) {
    # compute NMF
    nmf <- nfm_mult_upd(X, K, missing_idx = miss_idx, maxit=800)
    print(paste("Computed nmf, final dist is", nmf$dst))
    X_hat[K-k1+1,,] <- nmf$res
}


In [ ]:
# 3: weighted reconstruction
d = array(0, dim = c(kN-k1+1))
for (K in 1:(kN-k1+1)) {
    # Reconstruction error based on non missing values
    d[K] <- sum(abs(X_hat[K,,][non_miss_idx] - X[non_miss_idx]))/nrow(non_miss_idx)
}

X_hat_f <- matrix(0, nrow = nrow(X), ncol = ncol(X))
denum <- 0
for (K in 1:(kN-k1+1)) {
    # Reconstruction matrix
    X_hat_f <- X_hat_f + exp(-d[K])*X_hat[K,,]
    denum <- denum + exp(-d[K])
}
X_hat_f <- X_hat_f / sum(exp(-d))

In [ ]:
# Replace in the feature set
X[miss_idx] <- X_hat_f[miss_idx]
features_set[, numeric_variables_idx] <- X

## Factor variables

Now that we managed numerical variabels we can work on encoding the categorical ones which we did not work on yet.

Getting back to the columns containing many missing values, we use one hot encoding and create a category "missing", considering that not having this piece of information could be meaning something.

In [ ]:
useful_nas <- which(sapply(test_set_features, function(x) sum(is.na(x))) > 10000)

library(fastDummies)
features_set <- dummy_cols(features_set, 
                           select_columns = names(useful_nas),
                           remove_selected_columns = T)

## Change NAs to 0 for the new one hot encoded columns
replace_na_with_0<-function(vec) {
    vec[is.na(vec)]<-0
    vec
}

useful_nas_feat_idx <- c(grep("health_insurance_*", colnames(features_set)))
useful_nas_feat_idx <- c(useful_nas_feat_idx, grep("employment_industry*", colnames(features_set)))
useful_nas_feat_idx <- c(useful_nas_feat_idx, grep("employment_occup*", colnames(features_set)))

features_set[,useful_nas_feat_idx] <- replace_na_with_0(features_set[,useful_nas_feat_idx])

The remaining columns will be treated the same.

In [ ]:
factor_variables<-which(sapply(features_set[1,],class)=="factor")
factor_variables
factor_col_names <- colnames(factor_variables)

features_set_int_encoded <- dummy_cols(features_set,
                                       select_columns = factor_col_names,
                                       ignore_na = F,
                                       remove_selected_columns = T)


features_set_int_encoded <- replace_na_with_0(features_set_int_encoded)

tr_set_enc <- features_set_int_encoded[tr_indexes,]
ts_set_enc <- features_set_int_encoded[ts_indexes,]

summary(features_set_int_encoded)

# A RETIRER ?

We can choose several variables that should influence the output variables in our opinion and according to the scientific articles we have read.
### Pas hésiter à les mettre quand on en aura hihi
Mais selon moi, "marital_status", "rent_or_own", "education" et "employment_status" devraient pas trop influencer. Et "employment_industry" & "employment_occupation" sont des random short strings hyper nombreux donc pas convaincue que ça soit très utile.

In [ ]:
# We will need dummies package to transform string values with one-hot-encoding.
# install.packages('dummies')
library(dummies)

In [ ]:
# We keep only some factor variables
variables_to_keep<-c("age_group","race","sex","income_poverty","hhs_geo_region","census_msa")
data_factor_onehot <- dummy.data.frame(data_factor[,variables_to_keep], sep="_")

In [ ]:
dim(data_factor_onehot)

In [ ]:
data_factor_onehot[1:2,]

In [ ]:
data_preprocessed_extended<-cbind(data_preprocessed,data_factor_onehot)
dim(data_preprocessed_extended)

On peut passer au traitement des missing values. Pour le moment, j'implémente la même chose que dans les TP (on remplace par la mean value) mais on peut 100% faire plus de recherches et remplacer par autre chose. J'ai vu que la median value était souvent utilisée.

In [ ]:
replace_na_with_mean_value<-function(vec) {
    mean_vec<-mean(vec,na.rm=T)
    vec[is.na(vec)]<-mean_vec
    vec
}

In [ ]:
data_preprocessed_extended<-data.frame(apply(data_preprocessed_extended,2,replace_na_with_mean_value))
summary(data_preprocessed_extended)

In [ ]:
dim(na.omit(data_preprocessed_extended))

A ce stade, on a un set de data avec seulement des variables numériques qui ont du sens d'être là (selon moi hihi) et plus de missing values donc il manque surtout feature engineering. On pourra ensuite faire feature selection avec une mesure de la corrélation entre input et output.

## Feature engineering


## Feature selection


# Model selection

## Model 1
We are going to use the simple linear model first to see what kind of accuracy we can get. 

In [ ]:
# Normalement ce sera dans le préprocessing mais pas encore donc je laisse les lignes ici.
training_features <- read.csv("tr_set_imputed.csv")[, -c(1)]
summary(training_features)
training_set_labels <- read.csv("training_set_labels.csv", stringsAsFactors = T)
label_1 <- training_set_labels[,"h1n1_vaccine"]
label_2 <- training_set_labels[, "seasonal_vaccine"]
labels <- cbind(h1n1_vaccine=label_1, seasonal_vaccine=label_2)

testing_features <- read.csv("ts_set_imputed.csv")[, -c(1)]
feature_set <- rbind(training_features, testing_features)
feature_set <- feature_set[, -c(1)] # remove resp id
pca <- prcomp(feature_set)
summary(pca)

pca_feat_set <- pca$x[,1:54]

# install.packages("caret")
library("caret")

ss <- preProcess(as.data.frame(pca_feat_set), method=c("range"))
pca_feat_set <- predict(ss, as.data.frame(pca_feat_set))


pca_tr_set <- pca_feat_set[1:26707,]
pca_ts_set <- pca_feat_set[26708:53415,]
total_train_set <- cbind(pca_tr_set, labels)
summary(total_train_set)

Before making the prediction for the given test values, it is important to check that the model fits by doing a cross-validation and calculating the AUC and ROC. First, the cross-validation with the multivariate linear model.

In [ ]:
k = 10
accuracy_vec_h1n1 <- array(0,k)
accuracy_vec_seasonal <- array(0,k)
threshold <- 0.5

# 1. Shuffle the dataset randomly.
vaccine_idx <- sample(1:nrow(train_data_set))

# 2. Split the dataset into k groups
max <- ceiling(nrow(train_data_set)/k)
splits <- split(vaccine_idx, ceiling(seq_along(vaccine_idx)/max))

# 3. For each unique group:
for (i in 1:k){
  #3.1 Take the group as a hold out or test data set
  test_data <- train_data_set[splits[[i]],]
  
  #3.2 Take the remaining groups as a training data set
  train_data <- train_data_set[-splits[[i]],]
  print(paste("[INFO] - Training set size:",dim(train_data)[1],"- Testing set size",dim(test_data)[1]))
  
  #3.3 Fit a model on the training set and evaluate it on the test set
  model <- lm(cbind(h1n1_vaccine, seasonal_vaccine) ~ ., data=train_data)
  Y_pred <- predict(model,test_data[,-targets])
  Y_h1n1 <- test_data[,targets[2]]
  Y_seasonal <- test_data[,targets[1]]
  
  #3.4 Store the prediction of the tree (2 is to take only the P(Y="spam"|x))
  Y_hat <- ifelse(Y_pred > threshold,1,0) 
  # Need one confusion matrix for h1n1 and one for seasonal
  confusion_matrix_h1n1 <- table(Y_hat[,1],Y_h1n1)
  confusion_matrix_seasonal <- table(Y_hat[,2],Y_seasonal)
  
  #3.5 Retain the evaluation score and discard the model
  accuracy_vec_h1n1[i] = (confusion_matrix_h1n1[1,1]+confusion_matrix_h1n1[2,2])/sum(confusion_matrix_h1n1)
  misclassification_rate = 1 - accuracy_vec_h1n1[i]
  print(paste("[INFO] - Misclassification rate h1n1 -",i,"fold:",misclassification_rate))
  
  accuracy_vec_seasonal[i] = (confusion_matrix_seasonal[1,1]+confusion_matrix_seasonal[2,2])/sum(confusion_matrix_seasonal)
  misclassification_rate = 1 - accuracy_vec_seasonal[i]
  print(paste("[INFO] - Misclassification rate seasonal -",i,"fold:",misclassification_rate))
  
}

#4. Summarize the skill of the model using the sample of model evaluation scores
print(paste("[INFO] - Mean misclassification rate h1n1:",1-mean(accuracy_vec_h1n1)))
print(paste("[INFO] - Mean misclassification rate seasonal:",1-mean(accuracy_vec_seasonal)))

This gives us the misclassifaction rate for each of the targets.
Then, the ROC and AUC for each of the target.

In [ ]:
# Separation of the train in two groups in order to calculate the error between prediction and reality.
vaccine_idx <- sample(1:nrow(total_train_set))
half_split <- floor(nrow(total_train_set)/2)
train_data_set <- total_train_set[vaccine_idx[1:half_split],]
test_data <- total_train_set[vaccine_idx[(half_split+1):nrow(total_train_set)],]
target_idx <- ncol(train_data_set)
targets <- c(target_idx, target_idx-1)

# Linear model & prediction
model <- lm(cbind(h1n1_vaccine, seasonal_vaccine)~., data=train_data_set)
Y_pred <- predict(model,test_data[,-targets])

In [ ]:
# ROC for seasonal vaccine
thresholds <- seq(0,0.99,0.05)
FPR <- c()
TPR <- c()

for(threshold in thresholds){
  Y_hat <- ifelse(Y_pred > threshold,1,0) 
  confusion_matrix <- table(Y_hat[,2],test_data[,targets[1]])
  
  if(dim(confusion_matrix)[1] < 2){ 
    if(rownames(confusion_matrix) == 0){
      confusion_matrix <- rbind(confusion_matrix,c(0,0))
      rownames(confusion_matrix)[2] <- 1
    }
    if(rownames(confusion_matrix) == 1){
      confusion_matrix <- rbind(c(0,0),confusion_matrix)
      rownames(confusion_matrix)[1] <- 0
    }
  }
  
  FP <- confusion_matrix[2,1]
  TP <- confusion_matrix[2,2]
  N_N <- sum(confusion_matrix[,1]) # Total number of 0's
  N_P <- sum(confusion_matrix[,2]) # Total number of 1's
  
  FPR <- c(FPR,FP/N_N)
  TPR <- c(TPR,TP/N_P)
}

plot.new()
plot(FPR,TPR)
lines(FPR,TPR,col="blue")
lines(thresholds,thresholds,lty=2)
title("ROC Curve for seasonal vaccine")
AUC <- sum(abs(diff(FPR)) * (head(TPR,-1)+tail(TPR,-1)))/2
AUC

In [ ]:
# ROC for h1n1 vaccine
thresholds <- seq(0,0.99,0.05)
FPR <- c()
TPR <- c()

for(threshold in thresholds){
  Y_hat <- ifelse(Y_pred > threshold,1,0) 
  confusion_matrix <- table(Y_hat[,1],test_data[,targets[2]])
  
  if(dim(confusion_matrix)[1] < 2){ 
    if(rownames(confusion_matrix) == 0){
      confusion_matrix <- rbind(confusion_matrix,c(0,0))
      rownames(confusion_matrix)[2] <- 1
    }
    if(rownames(confusion_matrix) == 1){
      confusion_matrix <- rbind(c(0,0),confusion_matrix)
      rownames(confusion_matrix)[1] <- 0
    }
  }
  
  FP <- confusion_matrix[2,1]
  TP <- confusion_matrix[2,2]
  N_N <- sum(confusion_matrix[,1]) # Total number of 0's
  N_P <- sum(confusion_matrix[,2]) # Total number of 1's
  
  FPR <- c(FPR,FP/N_N)
  TPR <- c(TPR,TP/N_P)
}
AUC <- sum(abs(diff(FPR)) * (head(TPR,-1)+tail(TPR,-1)))/2
AUC

plot.new()
plot(FPR,TPR)
lines(FPR,TPR,col="blue")
lines(thresholds,thresholds,lty=2)
title("ROC Curve for h1n1 vaccine")

Now that the linear model is validated, it can be used to predict the outcome with the test set.

In [ ]:
model <- lm(cbind(h1n1_vaccine, seasonal_vaccine)~., data=total_train_set)
Y_pred <- predict(model,pca_ts_set)

## Model 2
Then we use the randomforest package to make another model.

In [ ]:
install.packages("randomForest")

In [ ]:
library("randomForest")
n_trees <- 5
n_trees
accuracy_vec <- array(0,n_trees)
accuracy_vec

vaccine_idx <- sample(1:nrow(total_train_set))
half_split <- floor(nrow(total_train_set)/2)
half_split

for (i in 1:n_trees){ print(i)
    #3.1 Take the first half of the dataset as a training data set
    train_data <- total_train_set[vaccine_idx[1:half_split],]

    #3.2 Take the second half of the dataset as a hold out or test data set
    test_data <- total_train_set[vaccine_idx[(half_split+1):nrow(total_train_set)],]
    
    model <- randomForest(x=train_data[,-c(labels)],
                          y=as.factor(train_data[,c(labels)]),
                          xtest=test_data[,-c(labels)],
                          ytest=as.factor(test_data[,c(labels)]),
                          ntree=i)
    
    accuracy_vec[i] = (model$test$confusion[1,1]+model$test$confusion[2,2])/sum(model$test$confusion)
    }

plot(accuracy_vec,main = "Number of trees influence",xlab = "Nbr of trees",ylab = "Classification rate") 

In [ ]:
# Ne pas décommenter car pas sûre que ça fonctionne
#library("randomForest")
# Create the forest.
#output.forest <- randomForest(formula=h1n1_vaccine+seasonal_vaccine~., data=total_train_set)

# View the forest results.
#print(output.forest) 

# Importance of each predictor.
#print(importance(fit,type = 2)) 
#help randomForest()

## Model 3

#### Example of simple equation
\begin{equation}
e = mc^2
\end{equation}

#### Example of matrix equation - Cross product formula:

\begin{equation*}
\mathbf{V}_1 \times \mathbf{V}_2 =  \begin{vmatrix}
\mathbf{i} & \mathbf{j} & \mathbf{k} \\
\frac{\partial X}{\partial u} &  \frac{\partial Y}{\partial u} & 0 \\
\frac{\partial X}{\partial v} &  \frac{\partial Y}{\partial v} & 0
\end{vmatrix}
\end{equation*}

#### Example of multiline equation - The Lorenz Equations:

\begin{align}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{align}

#### Example of Markdown Table:

| This | is   |
|------|------|
|   a  | table|


# Alternative models





# Conclusions